In [1]:
# Weights and Bias to version the dataset and experiment tracking
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 1.8 MB 141 kB/s eta 0:00:011
     |████████████████████████████████| 145 kB 104 kB/s eta 0:00:01
     |████████████████████████████████| 181 kB 185 kB/s eta 0:00:01
     |████████████████████████████████| 63 kB 260 kB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=5e5f25e2626fe8a26ef52d99626b9099709f5bd42285a8daff38799aeb0e3808
  Stored in directory: /tmp/pip-ephem-wheel-cache-2t677090/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=16607ea061fc31b59698d0fe0e197071df395f46d9b98e128c17e9d5bc8e30e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-2t677090/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built promise pathtools


In [2]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 292 kB 4.9 MB/s eta 0:00:01


In [3]:
%pip install scispacy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 44 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 46 kB/s eta 0:00:011
     |████████████████████████████████| 13.4 MB 63 kB/s eta 0:00:011
     |████████████████████████████████| 188 kB 85 kB/s eta 0:00:01
  Attempting uninstall: pybind11
    Found existing installation: pybind11 2.9.1
    Uninstalling pybind11-2.9.1:
      Successfully uninstalled pybind11-2.9.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 120.2 MB 1.8 MB/s eta 0:00:012  |█▏                              | 4.4 MB 824 kB/s eta 0:02:21     |█▍                              | 5.2 MB 824 kB/s eta 0:02:20     |██▋                             | 9.9 MB 3.7 MB/s eta 0:00:30     |████████▉                       | 33.1 MB 7.9 MB/s eta 0:00:12     |█████████                       | 33.4 MB 7.9 MB/s eta 0:00:12     |████████████▊                   | 47.8 MB 2.2 MB/s eta 0:00:34     |█████████████▌                  | 50.6 MB 3.5 MB/s eta 0:00:20     |███████████████                 | 56.2 MB 14.6 MB/s eta 0:00:05     |█████████████████▍              | 65.5 MB 7.5 MB/s eta 0:00:08     |█████████████████████▏          | 79.7 MB 7.6 MB/s eta 0:00:06     |██████████████████████▉         | 85.7 MB 4.6 MB/s eta 0:00:08     |████████████████████████████▋   | 107.3 MB 4.9 MB/s eta 0:00:03     |█████████████████████████████   | 109.0

In [5]:
import re
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import spacy
import string

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme(style="darkgrid")
import warnings
warnings.filterwarnings("ignore")

In [9]:
import scispacy
import wandb

In [7]:
nlp = spacy.load("en_core_sci_md")

In [10]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
import json
import random

In [13]:
# Label encoder to encode class labels
class LabelEncoder(object):
    """Encode labels into unqiue ids/integers"""
    def __init__(self, class_to_index={}):
        self.class_to_index = class_to_index or {}
        self.index_to_class = {v:k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        
    def __len__(self):
        return len(self.class_to_index)
    
    def __str__(self):
        return f"<LabelEncoder(num_classes={len(self)})>"
    
    def fit(self, y):
        classes = np.unique(y)
        for i, class_ in enumerate(classes):
            self.class_to_index[class_] = i
        self.index_to_class = {v:k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        return self
    
    def encode(self, y):
        encoded = np.zeros(len(y), dtype=int)
        for i, item in enumerate(y):
            encoded[i] = self.class_to_index[item]
        return encoded
    
    def decode(self, y):
        classes = []
        for i, item in enumerate(y):
            classes.append(self.index_to_class[item])
            
        return classes
    
    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {"class_to_index": self.class_to_index}
            json.dump(contents, fp, indent=4, sort_keys=False)
    
    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)

In [14]:
label_encoder = LabelEncoder.load("../../data_artifacts/label_encoder.json")

In [42]:
train_df = pd.read_parquet("../../dataset/v1.0/train/train.parquet")
val_df = pd.read_parquet("../../dataset/v1.0/val/val.parquet")
test_df = pd.read_parquet("../../dataset/v1.0/test/test.parquet")

In [46]:
# Feature and target
X_train = train_df[['transcription']]
y_train = train_df.specialty

X_val =  val_df[['transcription']]
y_val = val_df.specialty

X_test = test_df[['transcription']]
y_test = test_df.specialty

In [47]:
# Encode all our labels
y_train = label_encoder.encode(y_train)

y_val = label_encoder.encode(y_val)

y_test = label_encoder.encode(y_test)

In [48]:
def set_seeds(seed=42):
    """Set seeds for reproducibility"""
    np.random.seed(seed)
    random.seed(seed)

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
set_seeds()

In [51]:
# saving raw X_test to compare later
X_test_raw = X_test

In [52]:
class SpacyVectorTransform(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([self.nlp(text).vector for text in X])

In [53]:
column_preprocessor = ColumnTransformer(
    [
        ('text_spacy', SpacyVectorTransform(nlp), 'transcription'),
    ],
    remainder='drop',
    n_jobs=1
)

In [54]:
pipeline = Pipeline([
    ('column_preprocessor', column_preprocessor),
    ('Tree', DecisionTreeClassifier())
])

In [55]:
X_train.shape

(3476, 1)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)
metrics = precision_recall_fscore_support(y_test, y_pred, average="weighted")
performance = {"precision": metrics[0], "recall": metrics[1], "f1": metrics[2]}
print(json.dumps(performance, indent=2))

In [ ]:
ml_model_run = wandb.init(project="Medical-Transcription", name="DecisionTree_SciSpacy")
ml_model_run.log({"precision": performance["precision"], "recall": performance["recall"], "f1-score": performance["f1"]})